## Set Up

In [45]:
import os
import sys
import pandas as pd
import logging
from datetime import datetime, date
from google.cloud import bigquery
from typing import List


# **** SETUP ****

# change to match your filesystem
DATA_DIR = "../data/garys_store/files/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "receipts_1nf.csv")
# change to match your gcloud project 
PROJECT_NAME = "deb-01-372112"
DATASET_NAME = "tickets"
DATA_DIR = "./data/"
DEFAULT_TICKETS_FILE = os.path.join(DATA_DIR, "tickets.json")

# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'ticket_info': {
        'table_name': 'ticket_info',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('eticket_num', 'string', mode='REQUIRED', max_length=128),
            bigquery.SchemaField('confirmation', 'string', mode='REQUIRED'),
            bigquery.SchemaField('ticket_date', 'string', mode='REQUIRED'),
            bigquery.SchemaField('price', 'float', mode='REQUIRED'),
            bigquery.SchemaField('seat', 'string', mode='REQUIRED'),
            bigquery.SchemaField('status', 'string', mode='REQUIRED'),
        ]
    }
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

[DEBUG][2023-01-15 11:25:04,850][1311598642:0051] : Creating bigquery client
[INFO ][2023-01-15 11:25:04,856][1311598642:0054] : Setup Completed


## Load Original Ticket File

In [40]:
import json 
# receipts data file name
filename = DEFAULT_TICKETS_FILE
logger.debug(f"attempting to process: {filename}")

# *** always perform checks first ***
# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# load into dataframe
with open('./data/tickets.json') as data_file:    
    data = [json.loads(line) for line in data_file]

df = pd.json_normalize(data)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

# *** always perform check at the end ***
# check schema: contains all expected columns?
expected_columns = ['eticket_num', 'confirmation', 'ticket_date', 'price', 'seat', 'status', 'origin', 'destination']
for col in expected_columns:
    assert col in list(df.columns), f"Data file missing required column: {col}"

# assign & remember tickets dataframe
tickets_df = df
display(tickets_df.head(n=10))

[DEBUG][2023-01-15 11:20:03,946][2291644546:0004] : attempting to process: ./data/tickets.json
[INFO ][2023-01-15 11:20:04,461][2291644546:0017] : loaded 4096 rows from: ./data/tickets.json


,eticket_num,confirmation,ticket_date,price,seat,status,airline.name,airline.iata,airline.icao,airline.callsign,...,passenger.last_name,passenger.gender,passenger.birth_date,passenger.email,passenger.street,passenger.city,passenger.state,passenger.zip,origin,destination
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,Hawaiian Airlines,HA,HAL,HAWAIIAN,...,Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,NaN,NaN
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,Wizz Air,W6,WZZ,WIZZ AIR,...,Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,01093,NaN,NaN
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,AirAsia,AK,AXM,ASIAN EXPRESS,...,Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,NaN,NaN
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,Xiamen Airlines,MF,CXA,XIAMEN AIR,...,Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,NaN,NaN
5,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,Air New Zealand,NZ,ANZ,NEW ZEALAND,...,Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,NaN,NaN
6,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,Jeju Air,7C,JJA,JEJU AIR,...,Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,NaN,NaN
7,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,American Airlines,AA,AAL,AMERICAN,...,Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN
8,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,...,Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,NaN,NaN
9,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,...,Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,NaN,NaN


## Look up airlines.iata, airports.iata, passenger.passenger_sk

In [41]:
import db_dtypes
# products table query
query = f"""
SELECT
  iata,
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.airlines`
"""
# query bigquery table into dataframe
aldf = client.query(query).to_dataframe()
# index by product_name for the join
aldf = aldf.set_index(keys='iata')
logger.info(f"query {len(aldf.index)} rows from airlines table")

# customers table query
query = f"""
SELECT
  iata
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.airports`
"""
# query bigquery table into dataframe
apdf = client.query(query).to_dataframe()
# index by full_name for the join
apdf = apdf.set_index(keys='iata')
logger.info(f"query {len(apdf.index)} rows from airports table")

# customers table query
query = f"""
SELECT
  passenger_sk,
  email
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.passengers`
"""
# query bigquery table into dataframe
pdf = client.query(query).to_dataframe()

# drop email if they already exist
#   this line will allow us to re-run this cell
df = df.drop(columns=['email_x', 'email_y'], errors='ignore')

logger.debug(f"preparing for join")
# join with the main dataframe to get product_id and customer_id
df = df.merge(aldf, left_on='airline.iata', right_on='iata', how='inner')
df = df.merge(apdf, left_on='origin.iata', right_on='iata', how='inner')
df = df.merge(pdf, left_on='passenger.email', right_on='email', how='inner')

df = df.rename(columns={'origin.iata':'airport_iata', 'airline.iata': 'airline_iata'})



logger.debug(f"dataframe:")

display(df.columns)
display(df)


[INFO ][2023-01-15 11:20:32,839][2399994865:0013] : query 48 rows from airlines table
[INFO ][2023-01-15 11:20:34,561][2399994865:0026] : query 386 rows from airports table
[DEBUG][2023-01-15 11:20:36,321][2399994865:0043] : preparing for join
[DEBUG][2023-01-15 11:20:36,453][2399994865:0053] : dataframe:


Index(['eticket_num', 'confirmation', 'ticket_date', 'price', 'seat', 'status',
       'airline.name', 'airline_iata', 'airline.icao', 'airline.callsign',
       'airline.country', 'origin.name', 'origin.city', 'origin.country',
       'airport_iata', 'origin.icao', 'origin.latitude', 'origin.longitude',
       'origin.altitude', 'origin.tz_timezone', 'destination.name',
       'destination.city', 'destination.country', 'destination.iata',
       'destination.icao', 'destination.latitude', 'destination.longitude',
       'destination.altitude', 'destination.tz_timezone',
       'passenger.first_name', 'passenger.last_name', 'passenger.gender',
       'passenger.birth_date', 'passenger.email', 'passenger.street',
       'passenger.city', 'passenger.state', 'passenger.zip', 'origin',
       'destination', 'passenger_sk', 'email'],
      dtype='object')

,eticket_num,confirmation,ticket_date,price,seat,status,airline.name,airline_iata,airline.icao,airline.callsign,...,passenger.birth_date,passenger.email,passenger.street,passenger.city,passenger.state,passenger.zip,origin,destination,passenger_sk,email
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,13a85d8fc10eae051b15620870562606,robert.brown.69@hotmail.com
1,362-056643-6144,G29QFS,2022-03-22,827.27,9I,active,Air China,CA,CCA,AIR CHINA,...,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,13a85d8fc10eae051b15620870562606,robert.brown.69@hotmail.com
2,758-670187-2738,SC0SPN,2022-03-24,681.66,15A,active,Southwest Airlines,WN,SWA,SOUTHWEST,...,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,13a85d8fc10eae051b15620870562606,robert.brown.69@hotmail.com
3,808-619797-0756,V8IWMD,2022-03-23,358.24,1E,active,Vietnam Airlines,VN,HVN,VIET NAM AIRLINES,...,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,13a85d8fc10eae051b15620870562606,robert.brown.69@hotmail.com
4,775-967173-2939,FGJ1JH,2022-03-21,718.29,12F,active,Allegiant Air,G4,AAY,ALLEGIANT,...,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,13a85d8fc10eae051b15620870562606,robert.brown.69@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,765-093296-8064,ZGX5YH,2022-03-23,588.21,4D,active,Sichuan Airlines,3U,CSC,SI CHUAN,...,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN,317729f16eb44e3255f91c8ec1fa768a,jennifer.garcia.50@gmail.com
3994,430-840430-2690,PWVNDV,2022-03-21,309.32,28J,active,Sriwijaya Air,SJ,SJY,SRIWIJAYA,...,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN,317729f16eb44e3255f91c8ec1fa768a,jennifer.garcia.50@gmail.com
3995,998-693749-7580,ZH3MXV,2022-03-24,742.64,21G,active,S7 Airlines,S7,SBI,SIBERIAN AIRLINES,...,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN,317729f16eb44e3255f91c8ec1fa768a,jennifer.garcia.50@gmail.com
3996,400-923907-0561,SQ9TRU,2022-03-24,288.58,17B,active,United Airlines,UA,UAL,UNITED,...,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN,317729f16eb44e3255f91c8ec1fa768a,jennifer.garcia.50@gmail.com


## Load fact table to BigQuery

In [46]:
# *** generic load function ***
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")




# get unique tickets
tickets = df[['eticket_num', 'confirmation', 'ticket_date', 'price', 'seat', 'status', 'passenger_sk', 'airport_iata', 'airline_iata']].drop_duplicates(keep='first')
logger.info(f"Preparing {len(tickets.index)} unique tickets to load to bigquery")
display(tickets.head(n=10))
# load to bigquery
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['ticket_info']['table_name']}"
schema = TABLE_METADATA['ticket_info']['schema']
load_table(tickets, client, table_name, schema)

[INFO ][2023-01-15 11:25:13,029][4150504507:0042] : Preparing 3998 unique tickets to load to bigquery


,eticket_num,confirmation,ticket_date,price,seat,status,passenger_sk,airport_iata,airline_iata
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,13a85d8fc10eae051b15620870562606,YUL,MU
1,362-056643-6144,G29QFS,2022-03-22,827.27,9I,active,13a85d8fc10eae051b15620870562606,MEM,CA
2,758-670187-2738,SC0SPN,2022-03-24,681.66,15A,active,13a85d8fc10eae051b15620870562606,PCL,WN
3,808-619797-0756,V8IWMD,2022-03-23,358.24,1E,active,13a85d8fc10eae051b15620870562606,GIG,VN
4,775-967173-2939,FGJ1JH,2022-03-21,718.29,12F,active,13a85d8fc10eae051b15620870562606,BHX,G4
5,134-616161-5293,VCKN9N,2022-03-24,588.26,17A,active,13a85d8fc10eae051b15620870562606,TRU,MF
6,729-622695-2358,084F1E,2022-03-21,309.85,2J,active,13a85d8fc10eae051b15620870562606,TRU,BC
7,904-530679-1980,V9XA2I,2022-03-24,442.82,16E,active,13a85d8fc10eae051b15620870562606,BEL,WN
8,735-457823-0201,61KO0Q,2022-03-21,893.04,2F,active,13a85d8fc10eae051b15620870562606,CTS,S7
9,733-511168-0357,H0ZOXI,2022-03-22,898.01,22J,active,13a85d8fc10eae051b15620870562606,CNX,PR


[INFO ][2023-01-15 11:25:13,057][4150504507:0030] : loading table: 'deb-01-372112.tickets.ticket_info'
[INFO ][2023-01-15 11:25:18,136][4150504507:0035] : loaded 3998 rows into deb-01-372112:tickets.ticket_info
